# Анализ тональности отзывов, неделя 3

## Соревнование по сентимент - анализу

В настоящем ноутбуке содержится решение для Kaggle. Основная идея здесь - максимально
простыми средствами преодолеть бейзлайн, не тратя время на микрооптимизации.

Импортируем все, что необходимо

In [1]:
import csv

from sklearn.feature_extraction.text import CountVectorizer

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import pandas as pd

Считаем обучающие и тестовые данные:

In [2]:
train_X = []
train_y = []

with open('products_sentiment_train.tsv') as tsvfile:
  rd = csv.reader(tsvfile, delimiter='\t')
  for row in rd:
    sentiment, label = row
    train_X.append(sentiment)
    train_y.append(int(label))

In [3]:
test_X = []
test_id = []

with open('products_sentiment_test.tsv') as tsvfile:
  rd = csv.reader(tsvfile, delimiter='\t')
  for row in rd:
    id_, sentiment = row
    test_X.append(sentiment)
    test_id.append(id_)
    
test_X = test_X[1:]
test_id = test_id[1:]

Зададим базовый конвеер:

In [4]:
pipe = Pipeline([
    ('vect',CountVectorizer()),
    ('clf', LogisticRegression())
])

и параметры для перебора. На практике, в случае данной задачи и выбранных компонент конвейера значимым фактором оказывается ngram_range.

In [37]:
parameters = {
    'vect__ngram_range':[(1,1),(1,2),(1,3),(2,3),(2,4)]
}

Для выбора наилучшего ngram_range воспользуемся сеточным перебором по 5 фолдам:

In [29]:
grid_search = GridSearchCV(pipe, parameters, cv=5)

In [30]:
grid_search.fit(train_X,train_y)

/Users/aleksandrbogoyavlenskiy/miniconda3/envs/py3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                            

Наилучший вариант получается для ngram_range=(1,2).

In [31]:
grid_search.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,
                          

Просто для интереса вывел оценку точности (она ниже итоговой на Kaggle):

In [32]:
scores = cross_val_score(grid_search.best_estimator_,train_X,train_y, cv=5)
print(scores)
print(round(scores.mean(),3))
print(round(scores.std(),3) )

/Users/aleksandrbogoyavlenskiy/miniconda3/envs/py3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[0.77306733 0.7675     0.77       0.76       0.78195489]
0.771
0.007


Подготовка файла для загрузки на Kaggle. Почему-то csv writer из модуля csv пишет странный и неверный файл. Чем с этим разбираться,  проще и быстрее оказалось создать DataFrame и использовать pandas.to_csv

In [33]:
preds = grid_search.best_estimator_.predict(test_X)

In [35]:
preds_subm = pd.DataFrame(data={'Id':test_id,'y':preds})

In [36]:
preds_subm.to_csv('subm02.csv',index=False,header=True)